In [1]:
import os
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import codecs
from os import path
import csv
import networkx as nx
import numpy as np
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from utils import get_raw_data, dump_prediction
from gensim.parsing.preprocessing import preprocess_documents
import unicodedata

data_path = '../../data'

In [4]:
train_data, test_data, y_train, G, train_hosts, tests_hosts  = get_raw_data(encoding='utf-8', errors="ignore")

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 560/560 [00:00<?, ?it/s]


In [5]:
train_data[0].split()

['#polepharma',
 '»',
 'flux',
 'polepharma',
 '»',
 'flux',
 'des',
 'commentaires',
 'alternate',
 'alternate',
 'polepharma',
 'polepharma',
 '\ufeff',
 'recherche_________',
 'submit',
 '(button)',
 'toggle',
 'navigation',
 'polepharma',
 '*',
 'polepharma',
 '+',
 'qui',
 'sommes-nous',
 '?',
 '+',
 'organigramme',
 '+',
 'les',
 'chiffres',
 'clés',
 '+',
 'un',
 'réseau',
 '*',
 'territoires',
 '+',
 'région',
 'centre-val',
 'de',
 'loire',
 '+',
 'région',
 'normandie',
 '+',
 'conseil',
 'départemental',
 'd’eure&loir',
 '+',
 'chartres',
 'metropole',
 '+',
 'metropole',
 'de',
 'rouen',
 '+',
 'agglo',
 'du',
 'pays',
 'de',
 'dreux',
 '+',
 'agglo',
 'evreux',
 'portes',
 'de',
 'normandie',
 '+',
 'agglo',
 'seine&eure',
 '+',
 'agglo',
 'de',
 'tours',
 'plus',
 '*',
 'performance',
 'industrielle',
 '+',
 'excellence',
 'opérationnelle',
 '+',
 'maintenance',
 '+',
 'packaging',
 '+',
 'supply',
 'chain',
 '+',
 'usine',
 'du',
 'futur',
 '*',
 'talents',
 '*',
 'relat

In [6]:
from gensim.models import Word2Vec, KeyedVectors

In [7]:
path_to_model = 'C:/Users/luism/Downloads/frwiki_20180420_300d.txt.bz2'
if 'model' not in globals():
    model = KeyedVectors.load_word2vec_format(path_to_model)
if not os.path.isfile('./model.gensim'):
    model.save("./model.gensim")

In [8]:
class DocumentVectorizer:
    def __init__(self, model):
        self.model = model
        
    def vecotrize_sentence(self, sentence):
        return np.mean([self.model[word] for word in sentence if word in self.model], axis=0)
    
    def vectorize_documents(self, documents):
        return np.array([self.vecotrize_sentence(doc) if doc else np.zeros((self.model.vector_size, )) for doc in documents])

In [12]:
import regex as re
clean_tokenize = lambda sentence: re.sub(r'\W+', ' ', sentence).split()

In [13]:
vect = DocumentVectorizer(model)

In [14]:
vectors = vect.vectorize_documents(
    tqdm(map(clean_tokenize, train_data), total=len(train_data)))


100%|███████████████████████████████████████████████████████████████████████████████████| 2125/2125 [01:01<00:00, 34.30it/s]

In [15]:
import tensorflow.keras as keras
# Add dropout/batch-norm
class SimpleMLP(keras.Model):

    def __init__(self, use_bn=False, use_dp=False, num_classes=10):
        super(SimpleMLP, self).__init__(name='mlp')
        self.use_bn = use_bn
        self.use_dp = use_dp
        self.num_classes = num_classes
    
        self.dense1 = keras.layers.Dense(128, activation='relu')
        self.dense2 = keras.layers.Dense(64, activation='relu')
        self.dense3 = keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)

In [16]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
lab = LabelEncoder()
onhot = OneHotEncoder()
y_train_index = lab.fit_transform(y_train)
y_train_one_hot = onhot.fit_transform(y_train_index[:,np.newaxis])

In [17]:
vectors.shape, y_train_one_hot.shape, y_train_index.shape

((2125, 300), (2125, 8), (2125,))

In [18]:
from sklearn.model_selection import train_test_split
x_train_, x_valid_, y_train_, y_valid_ = train_test_split(vectors, y_train_one_hot, test_size=0.15, shuffle= True, stratify=y_train_index)

In [19]:
model = SimpleMLP(num_classes=y_train_one_hot.shape[1])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(x=x_train_, y=y_train_.todense(), validation_data=(x_valid_, y_valid_.todense()), epochs=200, batch_size=64)

Train on 1806 samples, validate on 319 samples
Epoch 1/200
1806/1806 [==============================] - ETA: 6s - loss: 2.0420 - accuracy: 0.15 - 0s 192us/sample - loss: 1.8388 - accuracy: 0.2791 - val_loss: 1.7750 - val_accuracy: 0.3260
Epoch 2/200
1806/1806 [==============================] - ETA: 0s - loss: 1.7603 - accuracy: 0.31 - 0s 32us/sample - loss: 1.7371 - accuracy: 0.3283 - val_loss: 1.7205 - val_accuracy: 0.3480
Epoch 3/200
1806/1806 [==============================] - ETA: 0s - loss: 1.8115 - accuracy: 0.26 - 0s 33us/sample - loss: 1.6856 - accuracy: 0.3848 - val_loss: 1.6731 - val_accuracy: 0.3950
Epoch 4/200
1806/1806 [==============================] - ETA: 0s - loss: 1.5797 - accuracy: 0.48 - 0s 32us/sample - loss: 1.6173 - accuracy: 0.3948 - val_loss: 1.6178 - val_accuracy: 0.3824
Epoch 5/200
1806/1806 [==============================] - ETA: 0s - loss: 1.4430 - accuracy: 0.45 - 0s 31us/sample - loss: 1.5494 - accuracy: 0.4336 - val_loss: 1.5770 - val_accuracy: 0.4890
Ep

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
from sklearn.model_selection import train_test_split
indexes = np.arange(len(y_train))
idx_train_, idx_valid_, _, _ = train_test_split(
    indexes, y_train, test_size=0.15, shuffle= True, stratify=y_train_index)

In [29]:
rf_clf = RandomForestClassifier()

In [34]:
rf_clf.fit(vectors[idx_train_], np.array(y_train)[idx_train_])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [35]:
y_pred = rf_clf.predict(vectors[idx_valid_])

In [36]:
from sklearn.metrics import classification_report

In [37]:
print(classification_report(np.array(y_train)[idx_valid_], y_pred))

C:\Users\luism\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                         precision    recall  f1-score   support

       business/finance       0.49      0.64      0.56        94
     education/research       0.46      0.42      0.44        31
          entertainment       0.50      0.62      0.55        87
         health/medical       0.58      0.50      0.54        14
             news/press       0.00      0.00      0.00        12
politics/government/law       0.67      0.40      0.50        30
                 sports       0.00      0.00      0.00         7
           tech/science       0.60      0.41      0.49        44

               accuracy                           0.51       319
              macro avg       0.41      0.37      0.38       319
           weighted avg       0.50      0.51      0.50       319

